In [ ]:
%pip install matplotlib scikit-learn pandas

In [ ]:
import json
import pandas as pd
from pathlib import Path
import os

# Set paths
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'dataset')
train_json_path = os.path.join(base_dir, 'train', 'labels.json')
val_json_path = os.path.join(base_dir, 'validation', 'labels.json')
train_images_dir = os.path.join(base_dir, 'train', 'data')
val_images_dir = os.path.join(base_dir, 'validation', 'data')

def load_filtered_annotations(json_path, target_class='Plastic bag'):
    """Load and filter annotations for only the target class"""
    with open(json_path) as f:
        data = json.load(f)
    
    images = {img['id']: img for img in data['images']}
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    records = []
    for ann in data['annotations']:
        img = images[ann['image_id']]
        class_name = categories[ann['category_id']]
        
        if class_name == target_class: 
            records.append({
                'ImageID': Path(img['file_name']).stem,
                'LabelName': '/m/05gqfk',
                'XMin': ann['bbox'][0] / img['width'],
                'YMin': ann['bbox'][1] / img['height'],
                'XMax': (ann['bbox'][0] + ann['bbox'][2]) / img['width'],
                'YMax': (ann['bbox'][1] + ann['bbox'][3]) / img['height']
            })
    return pd.DataFrame(records)

# Load only plastic bag annotations
print(f"Loading training data from: {train_json_path}")
df_train = load_filtered_annotations(train_json_path)
print(f"Loading validation data from: {val_json_path}")
df_validation = load_filtered_annotations(val_json_path)

# Update paths for the notebook
data_path_validation = os.path.join(val_images_dir, '*.jpg')
data_path_train = os.path.join(train_images_dir, '*.jpg')
validation_images_path = val_images_dir
train_images_path = train_images_dir

# Verify
print("\nFiltered Data loaded successfully!")
print(f"Found {len(df_train)} plastic bag training instances")
print(f"Found {len(df_validation)} plastic bag validation instances")
print("\nSample training data:")
print(df_train.head())

In [ ]:
len(df_train)

In [ ]:
len(df_validation)

In [ ]:
import glob
counter = 0

img_paths = data_path_train
folder = glob.glob(img_paths)

for i in folder:
    counter+=1
print(counter)

In [ ]:
import glob
counter = 0

img_paths = data_path_validation
folder = glob.glob(img_paths)

for i in folder:
    counter+=1
print(counter)

In [ ]:
import glob


train_list_ids = []
validation_list_ids = []

def get_ids(split, print_this, to_append_list):
    try:
        if split == data_path_train:
            folder = glob.glob(data_path_train)
            img_ids = [Path(f).stem for f in folder]
        elif split == data_path_validation:
            folder = glob.glob(data_path_validation)
            img_ids = [Path(f).stem for f in folder]
        else:
            raise Exception("Invalid Parameter")
        
        if not img_ids:
            print(f"Warning: No images found in {split}")
        
        to_append_list.extend(img_ids)
        print(f"Found {len(img_ids)} images in {split}")
        print(print_this)
        return True
    except Exception as e:
        print(f"Error in get_ids: {str(e)}")
        return False

# Clear and reinitialize lists
train_list_ids = []
validation_list_ids = []

# Run with debugging
print("\nProcessing training images:")
success_train = get_ids(data_path_train, "Finished training images", train_list_ids)

print("\nProcessing validation images:")
success_val = get_ids(data_path_validation, "Finished validation images", validation_list_ids)

# Verify results
if success_train and train_list_ids:
    print(f"\nFirst training ID: {train_list_ids[0]}")
else:
    print("Failed to get training IDs")

if success_val and validation_list_ids:
    print(f"First validation ID: {validation_list_ids[0]}")
else:
    print("Failed to get validation IDs")



In [ ]:
get_ids(data_path_train, "I finished the task", train_list_ids)

In [ ]:
train_list_ids[0]

In [ ]:
get_ids(data_path_validation, "I finished the task", validation_list_ids)

In [ ]:
validation_list_ids[1]

In [ ]:
# Load and analyze class information from JSON annotations
def get_class_info(json_path):
    with open(json_path) as f:
        data = json.load(f)
    
    # Get all categories
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Count occurrences of each class
    class_counts = {}
    for ann in data['annotations']:
        class_name = categories[ann['category_id']]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
    
    return pd.DataFrame({
        'className': list(categories.values()),
        'Object': list(categories.values()),  # Duplicate for compatibility
        'Count': [class_counts.get(name, 0) for name in categories.values()]
    })

# Get class information for both datasets
train_classes_df = get_class_info(train_json_path)
validation_classes_df = get_class_info(val_json_path)



In [ ]:
# Filter for plastic bags (case insensitive)
plastic_bag_names = ['plastic bag', 'Plastic bag', 'plastic_bag']  # Common variations
train_identifer = train_classes_df[train_classes_df['className'].str.lower().isin([name.lower() for name in plastic_bag_names])]
validation_identifer = validation_classes_df[validation_classes_df['className'].str.lower().isin([name.lower() for name in plastic_bag_names])]

print("Training set class information:")
print(train_classes_df)
print("\nPlastic bags in training set:")
print(train_identifer)
print('-'*40)
print("Validation set class information:")
print(validation_classes_df)
print("\nPlastic bags in validation set:")
print(validation_identifer)

In [ ]:
import random

def random_row_from_df(split):
    dataframe = ""
    
    if split == "train":
        dataframe,ids_to_check = df_train, train_list_ids#517
    elif split == "validation":
        dataframe,ids_to_check = df_validation, validation_list_ids#9
    else:
        raise Exception("Invalid parameter, must be either train or validation")
    
    img_ids_len = len(ids_to_check)
    rand = random.randint(0,img_ids_len-1)
    id = ids_to_check[rand]
    print(id)
    r = dataframe.loc[(dataframe.ImageID == id) & (dataframe.LabelName == '/m/05gqfk')]
    print(r)

In [ ]:
random_row_from_df("train")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def visualize_random_image(split):
    if split == 'train':
        images_path = train_images_path
    elif split == 'validation':
        images_path = validation_images_path
    else:
        raise Exception("Invalid input parameter, must be either train or validation")
    
    images_paths = glob.glob(os.path.join(images_path, '*.jpg'))
    random_image = random.choice(images_paths)
    img = mpimg.imread(random_image)
    
    fig, ax = plt.subplots()
    ax.imshow(img)
    plt.title(f"Random {split} image: {Path(random_image).name}")
    plt.show()

visualize_random_image("train")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
from pathlib import Path

def visualize_many(from_num, to_num, dataset):
    """Visualize multiple images"""
    if dataset == 'train':
        image_ids = train_list_ids
        base_path = data_path_train.replace('*.jpg', '')
    elif dataset == 'validation':
        image_ids = validation_list_ids
        base_path = data_path_validation.replace('*.jpg', '')
    else:
        raise ValueError("Dataset must be 'train' or 'validation'")
    
    total_images = len(image_ids)
    print(f"\nVisualizing {dataset} set (has {total_images} total images)")
    
    # Validate requested range
    if from_num >= total_images:
        print(f"Start number {from_num} >= total images {total_images}")
        return
    to_num = min(to_num, total_images)
    
    # Calculate grid layout
    num_images = to_num - from_num
    columns = 4
    rows = (num_images + columns - 1) // columns
    
    # Create figure
    fig = plt.figure(figsize=(15, 5*rows))
    plt.suptitle(f"{dataset.capitalize()} Images {from_num}-{to_num-1}", y=1.02)
    
    # Display images
    for i in range(from_num, to_num):
        try:
            img_id = image_ids[i]
            img_path = os.path.join(base_path, f"{img_id}.jpg")
            
            if not os.path.exists(img_path):
                print(f"Image not found: {img_path}")
                continue
                
            img = mpimg.imread(img_path)
            ax = fig.add_subplot(rows, columns, i-from_num+1)
            ax.imshow(img)
            ax.set_title(f"ID: {img_id}")
            ax.axis('off')
            
            # Add bounding boxes if available
            if dataset == 'train':
                boxes = df_train[df_train['ImageID'] == img_id]
            else:
                boxes = df_validation[df_validation['ImageID'] == img_id]
                
            for _, row in boxes.iterrows():
                xmin = row['XMin'] * img.shape[1]
                xmax = row['XMax'] * img.shape[1]
                ymin = row['YMin'] * img.shape[0]
                ymax = row['YMax'] * img.shape[0]
                
                rect = plt.Rectangle(
                    (xmin, ymin), xmax-xmin, ymax-ymin,
                    linewidth=1, edgecolor='r', facecolor='none'
                )
                ax.add_patch(rect)
                
        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
    
    plt.tight_layout()
    plt.show()

# Example usage:
print("=== Training Images ===")
visualize_many(0, 8, "train")  # Show first 8 training images

print("\n=== Validation Images ===")
visualize_many(0, 6, "validation")  # Show first 6 validation images

In [ ]:
df_validation.loc[(df_validation['ImageID'] == 'ecd5fc22a65b8d32') & (df_validation.LabelName == '/m/05gqfk')]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os

# Use the first available image from the training set
if train_list_ids:
    image_id = train_list_ids[0]  # Get first training image ID
    image_path = os.path.join(data_path_train.replace('*.jpg', ''), f"{image_id}.jpg")
    
    # Open and display the image
    im = Image.open(image_path)
    fig, ax = plt.subplots()
    ax.imshow(im)
    
    # Print image info
    image_width, image_height = im.size
    print(f"Image: {image_path}")
    print(f"Dimensions: {image_width} x {image_height}")
    
    # Example bounding box coordinates (normalized)
    xmin, ymin = 0.377083, 0.778125  # Normalized coordinates
    xmax, ymax = 0.745833, 0.96875
    
    # Convert to pixel coordinates
    xmin_px = xmin * image_width
    xmax_px = xmax * image_width
    ymin_px = ymin * image_height
    ymax_px = ymax * image_height
    
    # Draw the bounding box
    rect = patches.Rectangle(
        (xmin_px, ymin_px), 
        xmax_px - xmin_px,  # width
        ymax_px - ymin_px,  # height
        linewidth=2, 
        edgecolor='red', 
        facecolor='none'
    )
    ax.add_patch(rect)
    
    plt.title(f"Bounding Box Example\nImage ID: {image_id}")
    plt.show()
else:
    print("No training images found. Please check your data_path_train and train_list_ids")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import random
import glob
from PIL import Image
from pathlib import Path

def visualize_bb(dataset):
    images_path = train_images_path if dataset == 'train' else validation_images_path if dataset == 'validation' else 0
    if images_path == 0:
        raise Exception("Invalid input parameter, must be either train or validation")
    
    # Add pattern for glob
    image_folder = os.path.join(images_path, '*.jpg')
    images_paths = glob.glob(image_folder)
    num_of_images = len(images_paths)
    if num_of_images == 0:
        raise ValueError(f"No images found in {images_path}. Check path and files.")
    
    random_int = random.randint(0, num_of_images -1)
    random_image = images_paths[random_int]
    #print(random_image)
    img = Image.open(random_image)
    id_of_image = Path(random_image).stem  # Use stem for ID (no slicing)
    
    df = df_train if dataset == 'train' else df_validation
    df_rows = df.loc[(df.ImageID == id_of_image) & (df.LabelName == '/m/05gqfk')]
    
    image_width, image_height = img.size
    
    fig ,ax = plt.subplots()
    ax.imshow(img)
    
    
    for index, row in df_rows.iterrows():
        print(row['XMin'], row['XMax'], row['YMin'], row['YMax'])
        
        xmin = row['XMin']
        xmax = row['XMax']
        ymin = row['YMin']
        ymax = row['YMax']
        
        new_xmin = xmin * image_width
        new_xmax = xmax * image_width
        new_ymin = ymin * image_height
        new_ymax = ymax * image_height
        print(new_xmin,new_xmax,new_ymin,new_ymax)
        
        
        width = new_xmax - new_xmin
        height = new_ymax - new_ymin
        
        rect = patches.Rectangle((new_xmin,new_ymin), width, height, linewidth = 1, edgecolor = 'r', facecolor = 'none')
        ax.add_patch(rect)
    plt.show()
    
visualize_bb("train")

In [ ]:
print(len(df_train))
print(len(df_validation))

In [ ]:
def clean_dataframe(df_for_train, df_for_validation):
    # Debug: Check DataFrame content
    print("df_train LabelName unique:", df_for_train['LabelName'].unique())
    print("df_validation LabelName unique:", df_for_validation['LabelName'].unique())
    print(f"Before processing: len(df_train) = {len(df_for_train)}, len(df_validation) = {len(df_for_validation)}")
    
    # No filtering needed since all rows are '/m/05gqfk'
    df_train_cleansed = df_for_train
    df_validation_cleansed = df_for_validation
    
    print(f"After processing: len(df_train_cleansed) = {len(df_train_cleansed)}, len(df_validation_cleansed) = {len(df_validation_cleansed)}")
    return df_train_cleansed, df_validation_cleansed

df_train_cleansed, df_validation_cleansed = clean_dataframe(df_train, df_validation)

In [ ]:
len(df_train_cleansed)

#We have 517 train images, but some of those images have multiple bb, and the 986 is the number
#of rows we have in our new df_train_cleansed, and each row represents one bbounding



In [ ]:
986/517

In [ ]:
len(df_validation_cleansed)

In [ ]:
df_validation_cleansed

In [ ]:
df_train_cleansed

In [ ]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(df_train_cleansed, test_size = 0.2, random_state = 42)


In [ ]:
train

In [ ]:
test

In [ ]:
788 + 198

In [ ]:
import os

os.makedirs("001.Plastic_bag/images/train", exist_ok=True)
os.makedirs("001.Plastic_bag/images/test", exist_ok=True)

In [ ]:
test_ids = test["ImageID"].values.tolist()


In [ ]:
len(test_ids)

In [ ]:
train_ids = train["ImageID"].values.tolist()

In [ ]:
len(train_ids)

In [ ]:
import shutil
import glob
from pathlib import Path
import os

# Dynamically set base_dir
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'dataset')

# Create directories
os.makedirs(os.path.join(base_dir, '001.Plastic_bag/images/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, '001.Plastic_bag/images/test'), exist_ok=True)

# Move/copy images for test
to_loop = os.path.join(base_dir, 'train/data/*.jpg')
folder = glob.glob(to_loop)
new_path_test = os.path.join(base_dir, '001.Plastic_bag/images/test')

for path in folder:
    id = Path(path).stem
    if id in test_ids:
        if id in train_ids:
            shutil.copy(path, f"{new_path_test}/{id}.jpg")
        else:
            shutil.copy(path, f"{new_path_test}/{id}.jpg")  # Copy to preserve source

# Move/copy images for train
new_path_train = os.path.join(base_dir, '001.Plastic_bag/images/train')
for path in folder:
    id = Path(path).stem
    if id in train_ids:
        shutil.copy(path, f"{new_path_train}/{id}.jpg")

# Ensure final_train_df and final_test_df are defined
final_train_df = train.copy()
final_test_df = test.copy()

# Update ImagePath in final_train_df and final_test_df
final_train_df['ImageID'] = final_train_df['ImagePath'].apply(lambda x: Path(x).stem)
final_test_df['ImageID'] = final_test_df['ImagePath'].apply(lambda x: Path(x).stem)
final_train_df['ImagePath'] = final_train_df['ImageID'].apply(lambda x: f"001.Plastic_bag/images/train/{x}.jpg")
final_test_df['ImagePath'] = final_test_df['ImageID'].apply(lambda x: f"001.Plastic_bag/images/test/{x}.jpg")

# Debug
print("Updated final_train_df ImagePath:", final_train_df['ImagePath'].unique())
print("Updated final_test_df ImagePath:", final_test_df['ImagePath'].unique())


In [ ]:
# Count images in test
path_to_test = os.path.join(base_dir, '001.Plastic_bag/images/test/*.jpg')
folder = glob.glob(path_to_test)
counter = 0
for i in folder:
    counter += 1
print("Test images count:", counter)


In [ ]:
# Count images in train
path_to_train = os.path.join(base_dir, '001.Plastic_bag/images/train/*.jpg')
folder = glob.glob(path_to_train)
counter = 0
for i in folder:
    counter += 1
print("Train images count:", counter)

In [ ]:
#orignally we had 517 images, but because some of them are needed in both the train and test we now
#have a total of 449 + 162 images

In [ ]:
449 + 162

In [ ]:
611 - 517

In [ ]:
import glob
from pathlib import Path

test_image_ids = []
test_folder = glob.glob('001.Plastic_bag/images/test/*.jpg')

for i in test_folder:
    id = Path(i).stem
    test_image_ids.append(id)

In [ ]:
print(len(test_image_ids))

In [ ]:
train_image_ids = []
train_folder = glob.glob('001.Plastic_bag/images/train/*.jpg')

for i in train_folder:
    id = Path(i).stem
    train_image_ids.append(id)

In [ ]:
len(train_image_ids)

In [ ]:
train_ids = final_train_df['ImagePath'].tolist()
test_ids = final_test_df['ImagePath'].tolist()

print("train_ids:", train_ids)
print("test_ids:", test_ids)

In [ ]:
len(train_df_ids)

In [ ]:
len(test_df_ids)

In [ ]:
train_both = set(train_df_ids).intersection(train_image_ids)
len(train_both)

In [ ]:
test_both = set(test_df_ids).intersection(test_image_ids)
len(test_both)

In [ ]:
# 222 2 5 0.000 0.4 0.3 0.9 0.44  path/to/image11.jpg
# 222 2 5 0.000 0.5 0.2 0.33 0.8 path/to/image11.jpg

In [ ]:
#index header_column label_width className xmin ymin xmax ymax path

In [ ]:
train_df = train.copy()
test_df = test.copy()

In [ ]:
train_df.rename(columns = {"LabelName": "className"}, inplace = True)
train_df

In [ ]:
test_df.rename(columns = {"LabelName": "className"}, inplace = True)
test_df

In [ ]:
print(len(train_df))#788

In [ ]:
print(len(test_df))# 198

In [ ]:
train_df["header_cols"] = 2
train_df["label_width"] = 5


In [ ]:
train_df

In [ ]:
test_df["header_cols"] = 2
test_df["label_width"] = 5


In [ ]:
test_df

In [ ]:
train_df["ImagePath"] = "001.Plastic_bag/images/train/"+train_df['ImageID']+ '.jpg'
test_df["ImagePath"] = "001.Plastic_bag/images/test/"+test_df['ImageID']+ '.jpg'

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df = train_df[['header_cols', 'label_width', 'className', 'XMin', 'YMin', 'XMax', 'YMax', 'ImagePath']]
test_df = test_df[['header_cols', 'label_width', 'className', 'XMin', 'YMin', 'XMax', 'YMax', 'ImagePath']]

In [ ]:
train_df

In [ ]:
pd.set_option("display.precision",4)

In [ ]:
final_train_df = train_df.copy()
final_train_df['className'] = "0.000"


In [ ]:
final_train_df

In [ ]:
final_test_df = test_df.copy()
final_test_df['className'] = "0.000"

In [ ]:
final_test_df

In [ ]:
import shutil
import os

# Move 'train.lst' and 'test.lst' to '001.Plastic_bag/' (create if needed)
os.makedirs('001.Plastic_bag', exist_ok=True)

if os.path.exists('train.lst'):
    shutil.move('train.lst', os.path.join('001.Plastic_bag', 'train.lst'))
else:
    print("train.lst not found, skipping move")

if os.path.exists('test.lst'):
    shutil.move('test.lst', os.path.join('001.Plastic_bag', 'test.lst'))
else:
    print("test.lst not found, skipping move")

In [ ]:
import glob

folder = glob.glob('001.Plastic_bag/images/test/*.jpg')
count = 0
for i in folder:
    count+=1
print(count)

In [ ]:
test_ids = final_test_df['ImagePath'].tolist()
print(len(test_ids))

In [ ]:
image_path = test_ids[0] if test_ids else ''
df_row = final_test_df.loc[final_test_df['ImagePath'] == image_path ]
df_row

In [ ]:
import csv
from pathlib import Path

final = []
idx = 0  # Sequential index
for id in test_ids:
    
    df_rows = final_test_df.loc[final_test_df['ImagePath'] == id ]
    
    length = len(df_rows)
    count = 1
    arr = [idx, 2, 5]
    
    for index,row in df_rows.iterrows():
        xmin = str(row['XMin'])
        ymin = str(row['YMin'])
        xmax = str(row['XMax'])
        ymax = str(row['YMax'])
        
        arr.extend(["0.000", xmin, ymin, xmax, ymax])
        
        if count == length:
            arr.append(f"test/{Path(id).stem}.jpg")  # Relative path
        count+=1
        
        
    final.append(arr)
    idx += 1
    
    
with open('test.lst', 'w', newline = '') as out:
    for row in final:
        writer = csv.writer(out, delimiter = '\t')
        writer.writerow(row)
        
final_test_df[final_test_df['ImagePath'] == '']

In [ ]:

final_test_df[final_test_df['ImagePath'] == test_ids[0] if test_ids else '']

In [ ]:
import csv
from pathlib import Path

final = []
idx = 0  # Sequential index
for id in train_ids:
    
    df_rows = final_train_df.loc[final_train_df['ImagePath'] == id ]
    
    length = len(df_rows)
    count = 1
    arr = [idx, 2, 5]
    
    for index,row in df_rows.iterrows():
        xmin = str(row['XMin'])
        ymin = str(row['YMin'])
        xmax = str(row['XMax'])
        ymax = str(row['YMax'])
        
        arr.extend(["0.000", xmin, ymin, xmax, ymax])
        
        if count == length:
            arr.append(f"train/{Path(id).stem}.jpg")  # Relative path
        count+=1
        
        
    final.append(arr)
    idx += 1
    
    
with open('train.lst', 'w', newline = '') as out:
    for row in final:
        writer = csv.writer(out, delimiter = '\t')
        writer.writerow(row)

In [ ]:
#Theory Lesson

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open(os.path.join(train_images_dir, '029cf7b6d1a98a8c.jpg'))

fig, ax = plt.subplots()

ax.imshow(im)

image_width, image_height = im.size
print(image_width, image_height) # 768 1024




xmin = 0.0 * image_width
xmax = 0.9968 * image_width
ymin = 0.1 * image_height
ymax = 0.7339 * image_height


width = xmax - xmin
height = ymax - ymin

rect = patches.Rectangle((xmin,ymin), width, height, linewidth = 1, edgecolor = 'r', facecolor = 'none')


ax.add_patch(rect)
plt.show()

In [ ]:
#Theory Lesson

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Use a valid image path from dataset (replace with actual if needed)
im = Image.open(os.path.join(train_images_dir, '004bd64410736c69.jpg'))
im = im.transpose(Image.Transpose.FLIP_LEFT_RIGHT)

fig, ax = plt.subplots()

ax.imshow(im)

image_width, image_height = im.size
print(image_width, image_height) # Expected: 1024 683 (adjusted for your image)

# Sample normalized coords from JSON for this image (flipped Plastic bag)
xmin_flipped_norm = 1 - 0.6611  # Flipped from original xmax_norm
xmax_flipped_norm = 1 - 0.2328  # Flipped from original xmin_norm
ymin_norm = 0.1186
ymax_norm = 0.7358

xmin = xmin_flipped_norm * image_width
xmax = xmax_flipped_norm * image_width
ymin = ymin_norm * image_height
ymax = ymax_norm * image_height


  
width = xmax - xmin
height = ymax - ymin

rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
ax.add_patch(rect)


  # Display the image
ax.imshow(im)
plt.show()

In [ ]:
# import shutil
# shutil.move('','')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import random
import glob
from PIL import Image
from pathlib import Path
import os

# Update paths to point to dataset/001.Plastic_bag/
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'dataset')
test_images_path = os.path.join(base_dir, '001.Plastic_bag/images/test')  # Directory path
train_images_path = os.path.join(base_dir, '001.Plastic_bag/images/train')  # Directory path

# Debug: Check if images exist
print("Train images:", os.listdir(train_images_path))
print("Test images:", os.listdir(test_images_path))

def visualize_transposed_bb(dataset):
    images_path = train_images_path if dataset == 'train' else test_images_path if dataset == 'test' else 0
    if images_path == 0:
        raise Exception("Invalid input parameter, must be either train or test")
    
    # Add glob pattern for images
    image_folder = os.path.join(images_path, '*.jpg')
    images_paths = glob.glob(image_folder)
    num_of_images = len(images_paths)
    if num_of_images == 0:
        raise ValueError(f"No images found in {images_path}. Check path and files.")
    
    random_int = random.randint(0, num_of_images - 1)
    random_image = images_paths[random_int]
    print(random_image)
    img = Image.open(random_image)
    img = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    id_of_image = Path(random_image).stem  # Use stem for ID
    
    df = final_train_df if dataset == 'train' else final_test_df
    # Update filter to match ImagePath format in final_*_df
    df_rows = df.loc[(df['ImagePath'] == f"dataset/001.Plastic_bag/images/{dataset}/{id_of_image}.jpg") & (df['className'] == '0.000')]
    
    image_width, image_height = img.size
    
    fig, ax = plt.subplots()
    ax.imshow(img)
    
    for index, row in df_rows.iterrows():
        print(row['XMin'], row['XMax'], row['YMin'], row['YMax'])
        
        xmin = row['XMin']
        xmax = row['XMax']
        ymin = row['YMin']
        ymax = row['YMax']
        
        new_xmin = xmin * image_width
        new_xmax = xmax * image_width
        new_ymin = ymin * image_height
        new_ymax = ymax * image_height
        print(new_xmin, new_xmax, new_ymin, new_ymax)
        
        # Flip the bb coordinates
        xmax_flipped = (image_width/2) - (new_xmin-(image_width/2))
        xmin_flipped = (image_width/2) - (new_xmax-(image_width/2))
        
        width = xmax_flipped - xmin_flipped
        height = new_ymax - new_ymin
        
        rect = patches.Rectangle((xmin_flipped, new_ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
    
visualize_transposed_bb("train")

In [ ]:
len(final_test_df)

In [ ]:


# Update paths to point to dataset/001.Plastic_bag/
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'dataset')
test_images_path = os.path.join(base_dir, '001.Plastic_bag/images/test')  # Directory path
train_images_path = os.path.join(base_dir, '001.Plastic_bag/images/train')  # Directory path

# Debug: Check if images exist
print("Train images:", os.listdir(train_images_path))
print("Test images:", os.listdir(test_images_path))

def augment_data(dataset):
    images_path = train_images_path if dataset == "train" else test_images_path if dataset == "test" else 0
    if images_path == 0:
        raise Exception("Invalid Input parameter")
    
    df = final_train_df if dataset == "train" else final_test_df
    
    temp_df = pd.DataFrame(columns=["header_cols","label_width","className","XMin","YMin","XMax","YMax","ImagePath"], dtype=object)
    counter = 0
    for index, row in df.iterrows():
        img_path = row["ImagePath"]  # Image path in the df
        id = Path(img_path).stem  # Use stem for ID
        im_path = os.path.join(images_path, f"{id}.jpg")  # Full image path
        img = Image.open(im_path)
        image_width, image_height = img.size
        img_flip = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        img_flip.save(os.path.join(images_path, f"flipped_{id}.jpg"))
        new_image_path = f"dataset/001.Plastic_bag/images/{dataset}/flipped_{id}.jpg"  # Full path in df
        
        xmin = row['XMin'] * image_width
        xmax = row['XMax'] * image_width
        ymin = row['YMin']
        ymax = row['YMax']
        
        # Get new coordinates for flipped bounding boxes
        new_xmin = ((image_width/2)-(xmin-(image_width/2))) / image_width
        new_xmax = ((image_width/2)-(xmax-(image_width/2))) / image_width
        
        temp_df.loc[counter] = [2, 5, "0.000", new_xmin, ymin, new_xmax, ymax, new_image_path]
        counter += 1
    
    df_merged = pd.concat([df, temp_df], ignore_index=True)

    # Save .lst in dataset/001.Plastic_bag/
    df_merged.to_csv(os.path.join(base_dir, f'001.Plastic_bag/{dataset}.lst'), sep="\t", float_format="%.4f", header=None)
    print(len(df))  # Original df
    print("augmented df length below")
    print(len(temp_df))

In [ ]:
folder = glob.glob(train_images_path)
counter = 0
for i in folder:
    counter+=1
print(counter)

In [ ]:
folder = glob.glob(test_images_path)
counter = 0
for i in folder:
    counter+=1
print(counter)

In [ ]:
len(final_train_df)

In [ ]:
len(final_test_df)

In [ ]:
augment_data("test")

In [ ]:
folder = glob.glob(test_images_path)
counter = 0
for i in folder:
    counter+=1
print(counter)

In [ ]:
162 * 2

In [ ]:
augment_data("train")

In [ ]:
folder = glob.glob(train_images_path)
counter = 0
for i in folder:
    counter+=1
print(counter)

In [ ]:
449*2

In [ ]:
im=Image.open(os.path.join(train_images_dir, '004bd64410736c69.jpg'))
w,h=im.size

# Sample: 0	2	5	0.000	0.2328	0.1186	0.6611	0.7358	001.Plastic_bag/images/train/004bd64410736c69.jpg


fig, ax = plt.subplots()
xmin = 0.2328
xmax = 0.6611
ymin = 0.1186
ymax = 0.7358

xmin=xmin*w
xmax=xmax*w
ymin=ymin*h
ymax=ymax*h


  
width=xmax-xmin
height=ymax-ymin

rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
ax.add_patch(rect)


  # Display the image
ax.imshow(im)
plt.show()

In [ ]:
#Theory Lesson

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Use a valid image path from dataset (replace with actual if needed)
im = Image.open(os.path.join(train_images_dir, '004bd64410736c69.jpg'))
im = im.transpose(Image.Transpose.FLIP_LEFT_RIGHT)

fig, ax = plt.subplots()

ax.imshow(im)

image_width, image_height = im.size
print(image_width, image_height) # Expected: 1024 683 (adjusted for your image)

# Sample normalized coords from JSON for this image (flipped Plastic bag)
xmin_flipped_norm = 1 - 0.6611  # Flipped from original xmax_norm
xmax_flipped_norm = 1 - 0.2328  # Flipped from original xmin_norm
ymin_norm = 0.1186
ymax_norm = 0.7358

xmin = xmin_flipped_norm * image_width
xmax = xmax_flipped_norm * image_width
ymin = ymin_norm * image_height
ymax = ymax_norm * image_height


  
width = xmax - xmin
height = ymax - ymin

rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
ax.add_patch(rect)


  # Display the image
ax.imshow(im)
plt.show()

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install mxnet
!{sys.executable} -m pip install distro numpy==1.23.5 --force-reinstall

In [ ]:
import distro
import os

if distro.id() == "debian":
    os.system("apt-get update")
    os.system("apt-get install ffmpeg libsm6 libxext6 -y")


In [ ]:
RESIZE_SIZE = 256
BASE_DIR = os.path.join(os.path.dirname(os.getcwd()), 'dataset/001.Plastic_bag/')

In [ ]:
!python3 ../tools/im2rec.py --resize $RESIZE_SIZE --pack-label test.lst $BASE_DIR/images/


In [ ]:
!python3 ../tools/im2rec.py --resize $RESIZE_SIZE --pack-label train.lst $BASE_DIR/images/